<a href="https://colab.research.google.com/github/muskanlalit18/Security-Application-using-Image-Classification/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1 Building the CNN

# Importing Keras packages and libraries

In [0]:
%tensorflow_version 1.14
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense






#Initializing the CNN


In [0]:

classifier = Sequential()



#Step 1 - Convolution


In [0]:

classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), input_shape = (128, 128, 3), activation = 'relu'))





#Step 2 - Pooling

In [0]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

#Adding multiple convolution layers

In [0]:

classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))


In [0]:

classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))


# Step 3 - Flattening

In [0]:

classifier.add(Flatten())


# Step 4 - Full connection

In [0]:

classifier.add(Dense(activation = 'relu', units = 128))
classifier.add(Dense(activation = 'sigmoid', units = 1))

# Compiling the CNN

In [0]:
classifier.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 Fitting the CNN to the images

# unzipping a file uploaded on the drive

In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '17SzuqDzvm9Uqx4GPqwSI0Yt8NebueeEy'})
download.GetContentFile('dataset.tar')

In [0]:
from google.colab import files

In [0]:
!unzip dataset.tar


# obtaining the dataset

In [0]:

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
                                                 'dataset/training_set',
                                                  target_size=(128, 128),
                                                  batch_size=64,
                                                  class_mode='binary',
                                                 shuffle=True)

test_set = test_datagen.flow_from_directory(
                                            'dataset/test_set',
                                            target_size=(128, 128),
                                            batch_size=64,
                                            class_mode='binary',
                                            shuffle=True)



# Fitting the model

In [0]:
history = classifier.fit_generator(
                         training_set,
                         samples_per_epoch = 6169,
                         epochs=100,
                         validation_data = test_set,
                         validation_steps = 1531)

# Plotting the results


In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.ylim(0.60, 0.99)
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Acc')
plt.plot(epochs_range, val_acc, label='Validation Acc')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)

plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [0]:
classifier.summary()

#  Displaying images (can be ignored while testing)


In [0]:
sample_training_images, _ = next(train_datagen)

In [0]:

# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
  plotImages(sample_training_images[:5])

# Evaluating

In [0]:
loss, acc = classifier.evaluate(test_set,  verbose = 0)
print(acc * 100)
print(loss)